<a href="https://colab.research.google.com/github/SanghunOh/study_machinglearning/blob/main/codes/sanghunoh/reports/Term_2_MLP_MPIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Connect Drive

In [1]:
path_root = '/content/drive/MyDrive/datas/'

# 작업 경로 설정
import os
os.chdir(path_root)
!pwd

/content/drive/.shortcut-targets-by-id/1tHYgEz3lQF-LmtFPXRgMBDh4UN54qeAs/datas


In [2]:
# !unzip -o ./mpie_30_shuffle.zip -d ./mpie_30_shuffle

In [3]:
!ls

dataCh4_7.mat		 model_0.png
dataCh4_7.zip		 model_None.png
digitimages_binary.zip	 model.png
history_0.png		 mpie_30_shuffle
history_1.png		 mpie_30_shuffle.zip
history_2.png		 number_images
history_3.png		 results_20222205052715.pickle
history_4.png		 results_20222205135214.pickle
history_5.png		 results_20222205144206.pickle
history_6.png		 results_MLP_20222805072718.pickle
history_None.png	 results_MLP_20222805072816.pickle
iris_shuffle.mat	 results_MLP_20222805075221.pickle
iris.zip		 results.pickle
matlab_iris_shuffle.mat


###Load Dataset

In [4]:
path = path_root + 'mpie_30_shuffle/'

import pandas as pd

def loadDatasetFromCSV(_datafilename, _labelfilename, label_column):
  _feature_csv = pd.read_csv(path + _datafilename, dtype=np.float32, header=None) # image features of train data

  _label_csv = pd.read_csv(path + _labelfilename, dtype=np.float32, header=None) # labels of train data
  _label = _label_csv[label_column] # get label you want

  return _feature_csv, _label

In [5]:
import numpy as np

In [6]:
def preprocessDataset(label_column):
  # train
  train_feature_reshape, train_label = loadDatasetFromCSV('Traindata.csv', 'Trainlabel.csv', label_column)
  # test
  validation_feature_reshape, validation_label = loadDatasetFromCSV('Testdata.csv', 'Testlabel.csv', label_column)

  return train_feature_reshape, train_label, validation_feature_reshape, validation_label

In [7]:
train_feature, train_label, validation_feature, validation_label = preprocessDataset(1)
type(train_feature), train_feature.shape, type(train_label), train_label.shape, validation_feature.shape, validation_label.shape

(pandas.core.frame.DataFrame,
 (18777, 1024),
 pandas.core.series.Series,
 (18777,),
 (5086, 1024),
 (5086,))

####check model

In [8]:
import tensorflow as tf
from tensorflow.keras import layers

class_cnt = len(np.unique(train_label))
# class_cnt

model = tf.keras.Sequential()
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, input_dim=1024, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(layers.BatchNormalization())
model.add(layers.Dense(class_cnt, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

_epochs = 10
model.fit(train_feature, train_label)

587/587 [==============================] - 6s 7ms/step - loss: 1.1214 - accuracy: 0.5999


In [9]:
# input_layer = layers.Input(shape=(1024,))  # input: 1024 nodes
# x = tf.keras.layers.Flatten(name='Flatten')(input_layer)
# x = layers.Activation('relu')(layers.Dense(256, input_dim=1024)(x))  # hidden: 256 nodes
# x = layers.BatchNormalization()(x)
# y = layers.Activation('softmax')(layers.Dense(class_cnt)(x))    # number of output nodes : class_no (it changes depend on label you selected)
# model = tf.keras.Model(x, y)
# model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

# model.fit(train_feature, train_label)

###MPL CNN with BatchNormalization

In [10]:
import numpy as np

class_cnt = len(np.unique(train_label))
class_cnt

6

In [11]:
import tensorflow as tf
from tensorflow.keras import layers

def make_model(_class_cnt, _hidden_layers=1, dense_count=256, _kernel_initializer='glorot_uniform', _activation='relu', batch_normal=False, epochs=2, label_class=0):
  _model = tf.keras.Sequential()
  _model.add(layers.Dense(dense_count, input_dim=1024, kernel_initializer=_kernel_initializer, activation=_activation)) 
  if batch_normal:
    _model.add(layers.BatchNormalization())
  for idx in range(_hidden_layers):
    _model.add(layers.Dense(dense_count, kernel_initializer=_kernel_initializer, activation=_activation)) 
    if batch_normal:
      _model.add(layers.BatchNormalization())
  _model.add(layers.Dense(_class_cnt, activation='softmax')) 

  _model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

  return _model
model = make_model(_class_cnt=6)    
# tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, expand_nested=True, show_layer_activations=True) #layer_range=?, 

###callback function for fit time

In [12]:
import time

class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
TimeHistory()

### fit model

In [16]:
rows = train_feature
_batch_size = int(len(list(rows)) / (len(list(rows))/5))   # Just Check Model params quickly
# _batch_size = len(list(rows))

def model_fit(_param, _dataset):
  tf.keras.backend.clear_session()

  (_train_feature, _train_label, _validation_feature, _validation_label) = _dataset
  _param['_class_cnt'] = len(np.unique(_train_label))
  print(_param['epochs'], _param['_class_cnt'])

  model = make_model(**_param)
  time_callback = TimeHistory()
  history = model.fit(_train_feature, _train_label, epochs=_param['epochs'], validation_data=(_validation_feature, _validation_label), callbacks=[time_callback], steps_per_epoch=_batch_size)
  # history = model.fit(_train_feature, _train_label, epochs=_epochs, callbacks=[time_callback])
  execution_time = sum(time_callback.times)
  return model, history, execution_time, _param
dataset = preprocessDataset(1)  
param = {'_hidden_layers':1, 'dense_count':256, '_kernel_initializer':'glorot_uniform', '_activation':'relu', 'batch_normal':False, 'epochs':2}
model_fit(param, dataset)  

2 6
Epoch 1/2
5/5 [==============================] - 1s 245ms/step - loss: 1.7378 - accuracy: 0.3391 - val_loss: 1.5062 - val_accuracy: 0.5387
Epoch 2/2
5/5 [==============================] - 1s 187ms/step - loss: 1.4434 - accuracy: 0.5674 - val_loss: 1.4548 - val_accuracy: 0.5387


(<keras.engine.sequential.Sequential at 0x7f090bc5c9d0>,
 2.25327467918396,
 {'_activation': 'relu',
  '_class_cnt': 6,
  '_hidden_layers': 1,
  '_kernel_initializer': 'glorot_uniform',
  'batch_normal': False,
  'dense_count': 256,
  'epochs': 2})

### fit with multi params

In [17]:
params = [
          {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
          {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':15},
          {'_hidden_layers':0, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':45},
          # {'_hidden_layers':5, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':5},
          # {'_hidden_layers':5, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':15},
          # {'_hidden_layers':5, 'dense_count':64, '_kernel_initializer':None, '_activation':'relu', 'batch_normal':False, 'epochs':45},
          ]

label_classes = [0, 3, 5]
for label in label_classes:
  dataset = preprocessDataset(label)
  results = list()          
  for idx, param in enumerate(params):
    param['label_class'] = label
    print('-'*5 +'[ '+ str(idx) + ' ]'+ '-'*5 + str(param))
    results.append(model_fit(param, dataset))


-----[ 0 ]-----{'_hidden_layers': 0, 'dense_count': 64, '_kernel_initializer': None, '_activation': 'relu', 'batch_normal': False, 'epochs': 5, 'label_class': 0}
5 30
Epoch 1/5
5/5 [==============================] - 1s 167ms/step - loss: 3.4730 - accuracy: 0.0373 - val_loss: 3.4444 - val_accuracy: 0.0401
Epoch 2/5
5/5 [==============================] - 0s 98ms/step - loss: 3.4316 - accuracy: 0.0409 - val_loss: 3.4135 - val_accuracy: 0.0427
Epoch 3/5
5/5 [==============================] - 1s 126ms/step - loss: 3.4048 - accuracy: 0.0415 - val_loss: 3.3924 - val_accuracy: 0.0395
Epoch 4/5
5/5 [==============================] - 0s 93ms/step - loss: 3.3859 - accuracy: 0.0414 - val_loss: 3.3768 - val_accuracy: 0.0409
Epoch 5/5
5/5 [==============================] - 0s 94ms/step - loss: 3.3718 - accuracy: 0.0433 - val_loss: 3.3648 - val_accuracy: 0.0440
-----[ 1 ]-----{'_hidden_layers': 0, 'dense_count': 64, '_kernel_initializer': None, '_activation': 'relu', 'batch_normal': False, 'epochs': 

In [18]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%Y%d%m%H%M%S")
print("date and time =", dt_string)

import pickle
# save
with open(f'results_MLP_{dt_string}.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

now = 2022-05-28 08:00:15.934979
date and time = 20222805080015
INFO:tensorflow:Assets written to: ram://88d1d9ab-61b8-4f2f-958b-e624094025aa/assets
INFO:tensorflow:Assets written to: ram://78d7c402-3f8c-4829-95f5-1703acbc9b7e/assets
INFO:tensorflow:Assets written to: ram://5b6483c6-7d32-4152-9a24-9183c3d06782/assets
